In [1]:
#import arcpy
import arcpy

In [2]:
#set overwrite output to true allowing for you to 
#overwrite output files if you 
#make a mistake
arcpy.env.overwriteOutput = True

In [4]:
#Set the folder location of the project
#YOU WILL NEED TO CHANGE FOLDER LOCATION
#FOR YOUR COMPUTER TO THE LOCATION THE DATA FILES ARE SAVED IN
folder = "C:/Users/Krwinkler/Desktop/project"

In [5]:
#we created a geodatabase to copy the data into. When I turn this in 
#all of the data will already be in a geodatabase to make it easier for
#you to test our code
#I'm not sure if you will have to not run this line or if it will copy
#from one gdb to the other
#THE FIRST PARAMITER WILL NEED TO BE CHANGE FOR YOUR COMPUTER
arcpy.management.CreateFileGDB("C:/Users/Krwinkler/Desktop/project", "Final_Project.gdb")

<Result 'C:/Users/Krwinkler/Desktop/project\\Final_Project.gdb'>

In [6]:
#set the workspace to the location of the geodatabase that we just made
#WILL NEED TO BE CHANGE FOR DIFFERENT COMPUTERS
arcpy.env.workspace = "C:/Users/Krwinkler/Desktop/project/Final_Project.gdb"

In [7]:
#set variable cafo_shp to folder + the location within the folder
#and shp file for cafos
cafo_shp = folder + "/CAFO_Data/Animal_Feeding_Operations.shp"

In [12]:
#set variable wells_shp to folder + the location within the folder
#and shp file for wells
wells_shp = folder + "/Wells_data/PWTS_Wells.shp"

In [9]:
#set a variable that we will copy the CAFO.shp file to in the new gdb
cafoFC = "cafo"

In [10]:
#set a variable that we will copy the wells.shp file to in the new gdb
wellsFC = "wells"

In [13]:
#this copies the wells shp file to the wellsFC variable and does this in
#the workspace with is our new gdb
arcpy.CopyFeatures_management(wells_shp, wellsFC)

<Result 'C:/Users/Krwinkler/Desktop/project/Final_Project.gdb\\wells'>

In [14]:
#this copies the Cafo shp file to the cafoFC variable and does this in
#the workspace with is our new gdb
arcpy.CopyFeatures_management(cafo_shp, cafoFC)

<Result 'C:/Users/Krwinkler/Desktop/project/Final_Project.gdb\\cafo'>

In [15]:
#set teh variable dem to the folder + the folder within it that 
#contains the DEM and the name of the DEM file
dem = folder + "/DEM/DEM_3m_I_52.img"

In [16]:
#this copies the raster to a new file called "dem" within the new gdb
arcpy.CopyRaster_management(dem,"dem")

<Result 'C:/Users/Krwinkler/Desktop/project/Final_Project.gdb\\dem'>

In [17]:
#we need to import these from arcpy to allow us to do 
#raster analysis
from arcpy import env
from arcpy.sa import *

In [ ]:
#the fill function fills any holes in the raster data
#outFill.save saves the output into the workspace
outFill = Fill("dem")
outFill.save("dem_fill")

In [19]:
#the FlowDirection function preforms a kind of raster analysis 
#that determines the direction of flow in the raster
#outFlowDirection.save saves the output file in the workspace
outFlowDirection = FlowDirection("dem_fill")
outFlowDirection.save("flow_dir")

In [20]:
#FlowAccumulation tool finds the canels that the water flows 
#into, this prevent there from being five small streams running 
#next to one another and instead combines them into one canel which 
#is more realistic
outFlowAccumulation = FlowAccumulation(outFlowDirection)

In [ ]:
#SnapPourPoints snaps the created streams and snaps them to points
#for this we use the wellsFC as the points we snap to
outSnapPour = SnapPourPoint(wellsFC, outFlowAccumulation, 100)
# outSnapPour.save saves the output file into the workspace 
outSnapPour.save("snappntwells")

In [ ]:
#Watershed tool takes the flow_dir, snap points and uses them
#to create a watershed
outWatershed = Watershed("flow_dir", "snappntwells", "VALUE")
# outWatershed.save saves the watershed to the workspace 
outWatershed.save("wells_watershed")

In [ ]:
#raster to polygon conversion is done on the output water shed
#to make the output watershed a polygon allowing us to use 
#summarize within
arcpy.RasterToPolygon_conversion("wells_watershed", "watershed_polygons", "NO_SIMPLIFY", "VALUE")

In [ ]:
#summariveWithin tool to get a count of the cafos within the watershed
arcpy.analysis.SummarizeWithin("watershed_polygons", "cafo", "watershed_CAFO_poly")

In [ ]:
#summariveWithin tool to get a count of the wells within the watershed
arcpy.analysis.SummarizeWithin("watershed_polygons", "wells", "watershed_wells_poly")